In [ ]:
# Install Dependencies

!pip uninstall -y numpy
!pip install --force-reinstall numpy==1.26.4

!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 sentence-transformers==2.2.2
!pip install pandas==2.0.0
!pip install transformers==4.41.0 scikit-learn==1.2.0
!pip install huggingface-hub==0.25.2
!pip install nltk==3.8.1 rouge-score==0.1.2 bert-score==0.3.13 -q
!pip install tqdm==4.66.5 -q
!pip install lime shap -q

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Setup and Imports

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BartForConditionalGeneration, BartTokenizer,
    DPRContextEncoder, DPRQuestionEncoder,
    DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer
)
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import os
from google.colab import drive
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import string
import nltk
import json
import pickle
import lime
import lime.lime_text
import shap

nltk.download('wordnet')
nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')

# Configuration
class Config:
    BASE_PATH = "/content/drive/MyDrive/LJMU-Datasets"
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BART_MODEL_NAME = "facebook/bart-base"
    DPR_CTX_MODEL_NAME = "facebook/dpr-ctx_encoder-single-nq-base"
    DPR_QUESTION_MODEL_NAME = "facebook/dpr-question_encoder-single-nq-base"
    BATCH_SIZE = 8
    MAX_EPOCHS = 3
    NUM_WORKERS = 4
    MAX_LENGTH = 256
    SUBSET_SIZE = 500
    HOTPOTQA_MAX_SAMPLES = 1000
    WIKIDATA_SUBSET_SIZE = 30000

CONFIG = Config()
print(f"Using device: {CONFIG.DEVICE}")

# Clear GPU memory
torch.cuda.empty_cache()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
Using device: cuda


In [ ]:
# Load Artifacts from Step 3 and Rebuild DataLoaders

import pickle
import os

save_path = '/content/drive/MyDrive/bert_retrieval_artifacts_v4'

# Define file paths (ensure correct naming and directory)
qa_train_path_v4 = os.path.join(CONFIG.BASE_PATH, "qa_train_v4.csv")
qa_val_path_v4 = os.path.join(CONFIG.BASE_PATH, "qa_val_v4.csv")
triple_train_path_v4 = os.path.join(CONFIG.BASE_PATH, "triple_train_v4.csv")

# Check if files exist, if not, attempt to locate or re-generate
if not os.path.exists(qa_train_path_v4) or not os.path.exists(qa_val_path_v4) or not os.path.exists(triple_train_path_v4):
    print("One or more dataset files not found. Attempting to locate or re-generate...")
    # Alternative path
    qa_train_path_alt = os.path.join(CONFIG.BASE_PATH, "qa_train.csv")
    qa_val_path_alt = os.path.join(CONFIG.BASE_PATH, "qa_val.csv")
    triple_train_path_alt = os.path.join(CONFIG.BASE_PATH, "triple_train.csv")

    if os.path.exists(qa_train_path_alt) and os.path.exists(qa_val_path_alt) and os.path.exists(triple_train_path_alt):
        print("Found alternative files without _v4 suffix. Using them instead.")
        qa_train_path_v4 = qa_train_path_alt
        qa_val_path_v4 = qa_val_path_alt
        triple_train_path_v4 = triple_train_path_alt
    else:
        print("Alternative files not found. Ensure Step 1 was run to generate qa_train_v4.csv, qa_val_v4.csv, and triple_train_v4.csv.")
        raise FileNotFoundError("Required dataset files not found.")

# Load raw datasets
qa_train_df_v4 = pd.read_csv(qa_train_path_v4)
qa_val_df_v4 = pd.read_csv(qa_val_path_v4)
triple_train_df_v4 = pd.read_csv(triple_train_path_v4)

# Split triple_train_df into train and validation sets (80/20)
triple_train_df_v4, triple_val_df_v4 = train_test_split(triple_train_df_v4, train_size=0.8, random_state=42)
print(f"Triple Train Size (Version 4): {len(triple_train_df_v4)}, Triple Val Size (Version 4): {len(triple_val_df_v4)}")

# Load all_candidates
with open(os.path.join(save_path, 'all_candidates_v4.pkl'), 'rb') as f:
    all_candidates = pickle.load(f)

# Define RetrievalDataset class
class RetrievalDataset(Dataset):
    def __init__(self, df: pd.DataFrame, bart_tokenizer: BartTokenizer, dpr_question_tokenizer: DPRQuestionEncoderTokenizer,
                 max_length: int = 256, task: str = "qa", candidate_objects: list = None):
        self.bart_tokenizer = bart_tokenizer
        self.dpr_question_tokenizer = dpr_question_tokenizer
        self.max_length = max_length
        self.task = task
        self.data = df
        self.candidate_objects = candidate_objects

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row["question"]
        context = row["context"]
        answer = row["answer"]

        if self.task == "qa":
            bart_input_text = f"question: {question} context: {context}"
        else:
            bart_input_text = f"complete the triple with the exact object: {question} context: {context}"
        bart_inputs = self.bart_tokenizer(
            bart_input_text,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )
        bart_labels = self.bart_tokenizer(
            answer,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        dpr_inputs = self.dpr_question_tokenizer(
            question,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        item = {
            "task": self.task,
            "bart_input_ids": bart_inputs["input_ids"].squeeze().long(),
            "bart_attention_mask": bart_inputs["attention_mask"].squeeze().long(),
            "bart_labels": bart_labels["input_ids"].squeeze().long(),
            "dpr_input_ids": dpr_inputs["input_ids"].squeeze().long(),
            "dpr_attention_mask": dpr_inputs["attention_mask"].squeeze().long(),
            "question": question,
            "context": context,  # Added context field
            "answer": answer
        }

        if self.task == "triple" and self.candidate_objects:
            label_idx = self.candidate_objects.index(answer) if answer in self.candidate_objects else -1
            item["label_idx"] = label_idx

        return item

# Create DataLoaders from raw data
bart_tokenizer = BartTokenizer.from_pretrained(CONFIG.BART_MODEL_NAME)
dpr_question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)

qa_train_dataset_v4 = RetrievalDataset(qa_train_df_v4, bart_tokenizer, dpr_question_tokenizer, task="qa", candidate_objects=all_candidates)
qa_val_dataset_v4 = RetrievalDataset(qa_val_df_v4, bart_tokenizer, dpr_question_tokenizer, task="qa", candidate_objects=all_candidates)
triple_train_dataset_v4 = RetrievalDataset(triple_train_df_v4, bart_tokenizer, dpr_question_tokenizer, task="triple", candidate_objects=all_candidates)
triple_val_dataset_v4 = RetrievalDataset(triple_val_df_v4, bart_tokenizer, dpr_question_tokenizer, task="triple", candidate_objects=all_candidates)

qa_train_loader_v4 = DataLoader(qa_train_dataset_v4, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)
qa_val_loader_v4 = DataLoader(qa_val_dataset_v4, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=CONFIG.NUM_WORKERS)
triple_train_loader_v4 = DataLoader(triple_train_dataset_v4, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)
triple_val_loader_v4 = DataLoader(triple_val_dataset_v4, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=CONFIG.NUM_WORKERS)

print(f"Created QA DataLoaders (Version 4): QA Train={len(qa_train_dataset_v4)}, QA Val={len(qa_val_dataset_v4)}")
print(f"Created Triple DataLoaders (Version 4): Triple Train={len(triple_train_dataset_v4)}, Triple Val={len(triple_val_dataset_v4)}")

# Load BART models and tokenizer
bart_qa_model = BartForConditionalGeneration.from_pretrained(CONFIG.BART_MODEL_NAME).to(CONFIG.DEVICE)
bart_qa_model.load_state_dict(torch.load(os.path.join(save_path, 'bart_qa_v4.pt')))
bart_triple_model = BartForConditionalGeneration.from_pretrained(CONFIG.BART_MODEL_NAME).to(CONFIG.DEVICE)
bart_triple_model.load_state_dict(torch.load(os.path.join(save_path, 'bart_triple_v4.pt')))
bart_tokenizer = BartTokenizer.from_pretrained(CONFIG.BART_MODEL_NAME)
bart_qa_model.eval()
bart_triple_model.eval()

# Load DPR models and tokenizers (post-RL fine-tuning from Step 3)
ctx_encoder_qa = DPRContextEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_ctx_encoder_rl_qa_v4")).to(CONFIG.DEVICE)
question_encoder_qa = DPRQuestionEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_question_encoder_rl_qa_v4")).to(CONFIG.DEVICE)
ctx_encoder_triple = DPRContextEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_ctx_encoder_rl_triple_v4")).to(CONFIG.DEVICE)
question_encoder_triple = DPRQuestionEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_question_encoder_rl_triple_v4")).to(CONFIG.DEVICE)
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(CONFIG.DPR_CTX_MODEL_NAME)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)
candidate_embeddings_qa = torch.load(os.path.join(save_path, 'dpr_candidate_embeddings_rl_qa_v4.pt')).to(CONFIG.DEVICE)
candidate_embeddings_triple = torch.load(os.path.join(save_path, 'dpr_candidate_embeddings_rl_triple_v4.pt')).to(CONFIG.DEVICE)

# Load sentence transformer
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

print("Artifacts loaded from Step 3 and DataLoaders rebuilt for Version 4.")

Triple Train Size (Version 4): 20908, Triple Val Size (Version 4): 5227
Created QA DataLoaders (Version 4): QA Train=294, QA Val=27
Created Triple DataLoaders (Version 4): Triple Train=20908, Triple Val=5227


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


Artifacts loaded from Step 3 and DataLoaders rebuilt for Version 4.


In [ ]:
# Define Helper Functions

# Normalize text for evaluation
def normalize_text(text: str) -> str:
    text = str(text).lower().strip()
    text = text.translate(str.maketrans("", "", string.punctuation))
    articles = {'a', 'an', 'the'}
    words = text.split()
    words = [word for word in words if word not in articles]
    return ' '.join(words)

# Compute BLEU score
def compute_bleu(generated: str, reference: str) -> float:
    return sentence_bleu([reference.split()], generated.split())

# Compute ROUGE-L score
def compute_rouge_l(generated: str, reference: str) -> float:
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    return scorer.score(reference, generated)['rougeL'].fmeasure

# Compute BERTScore
def compute_bertscore(generated: str, reference: str) -> float:
    return bert_score([generated], [reference], lang="en", verbose=False)[2].mean().item()

print("Helper functions defined.")

Helper functions defined.


In [ ]:
# Explainability with LIME and Custom SHAP

import gc

# Function to compute BART probabilities for a given text
def compute_bart_probs(texts, model, tokenizer, max_vocab_size=50265):  # BART's default vocab size
    if isinstance(texts, str):
        texts = [texts]
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=CONFIG.MAX_LENGTH
    )
    inputs = {k: v.to(CONFIG.DEVICE) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # Shape: (batch_size, seq_len, vocab_size)
        # Take the mean over the sequence length to get a single probability distribution per sample
        logits = logits.mean(dim=1)  # Shape: (batch_size, vocab_size)
        # Ensure consistent vocab size by padding or truncating
        current_vocab_size = logits.shape[-1]
        if current_vocab_size < max_vocab_size:
            padding = torch.zeros((logits.shape[0], max_vocab_size - current_vocab_size), device=logits.device)
            logits = torch.cat([logits, padding], dim=-1)
        elif current_vocab_size > max_vocab_size:
            logits = logits[:, :max_vocab_size]
        probs = torch.softmax(logits, dim=-1)
    return probs.cpu().numpy()

# Wrapper function for LIME to handle perturbed samples
def lime_bart_predict(texts, model, tokenizer):
    if not isinstance(texts, list):
        texts = [texts]
    probs_list = []
    for i in range(0, len(texts), CONFIG.BATCH_SIZE):
        batch_texts = texts[i:i + CONFIG.BATCH_SIZE]
        batch_probs = compute_bart_probs(batch_texts, model, tokenizer)
        probs_list.append(batch_probs)
        torch.cuda.empty_cache()
        gc.collect()
    probs = np.concatenate(probs_list, axis=0)
    return probs

# LIME Explainer (Optimized for memory usage)
def explain_with_lime(model, val_loader, task: str = "qa", num_samples: int = 5):
    print(f"Explaining {task} predictions with LIME...")
    explainer = lime.lime_text.LimeTextExplainer(class_names=["answer"])
    explanations = []
    model.eval()

    samples_processed = 0
    for batch in val_loader:
        if samples_processed >= num_samples:
            break
        references = batch["answer"]
        for i in range(len(batch["question"])):
            if samples_processed >= num_samples:
                break
            text_input = f"question: {batch['question'][i]} context: {batch['context'][i]}"
            explanation = explainer.explain_instance(
                text_input,
                lambda texts: lime_bart_predict(texts, model, bart_tokenizer),
                num_features=6,
                num_samples=50
            )
            explanations.append({
                "text": text_input,
                "reference": references[i],
                "explanation": explanation.as_list()
            })
            samples_processed += 1
            torch.cuda.empty_cache()
            gc.collect()

    return explanations

# Custom permutation-based SHAP approximation (Updated to handle inconsistent sequence lengths)
def custom_permutation_shap(text, model, tokenizer, num_permutations=50):
    print(f"Computing custom permutation SHAP for text: {text[:50]}...")
    model.eval()
    baseline_input = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=CONFIG.MAX_LENGTH
    )
    baseline_input = {k: v.to(CONFIG.DEVICE) for k, v in baseline_input.items()}
    with torch.no_grad():
        baseline_output = model(**baseline_input).logits
        # Average over sequence length to get a single probability distribution
        baseline_output = baseline_output.mean(dim=1)  # Shape: (1, vocab_size)
        baseline_probs = torch.softmax(baseline_output, dim=-1).cpu().numpy()[0]  # Shape: (vocab_size,)

    feature_importance = np.zeros(len(baseline_probs))
    words = text.split()
    if len(words) < 2:
        return feature_importance

    for _ in range(num_permutations):
        permuted_words = words.copy()
        np.random.shuffle(permuted_words)
        permuted_text = " ".join(permuted_words)
        permuted_input = tokenizer(
            permuted_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=CONFIG.MAX_LENGTH
        )
        permuted_input = {k: v.to(CONFIG.DEVICE) for k, v in permuted_input.items()}
        with torch.no_grad():
            permuted_output = model(**permuted_input).logits
            # Average over sequence length to get a single probability distribution
            permuted_output = permuted_output.mean(dim=1)  # Shape: (1, vocab_size)
            permuted_probs = torch.softmax(permuted_output, dim=-1).cpu().numpy()[0]  # Shape: (vocab_size,)
        importance = np.abs(baseline_probs - permuted_probs).mean()
        feature_importance += importance
        torch.cuda.empty_cache()
        gc.collect()

    return feature_importance / num_permutations

# SHAP Explainer (Custom permutation-based, optimized for memory)
def explain_with_shap(model, val_loader, task: str = "qa", num_samples: int = 5):
    print(f"Explaining {task} predictions with custom permutation SHAP...")
    model.eval()
    explanations = []

    samples_processed = 0
    for batch in val_loader:
        if samples_processed >= num_samples:
            break
        references = batch["answer"]
        for i in range(len(batch["question"])):
            if samples_processed >= num_samples:
                break
            text_input = f"question: {batch['question'][i]} context: {batch['context'][i]}"
            shap_values = custom_permutation_shap(text_input, model, bart_tokenizer, num_permutations=50)
            explanations.append({
                "text": text_input,
                "reference": references[i],
                "shap_values": shap_values.tolist()
            })
            samples_processed += 1
            torch.cuda.empty_cache()
            gc.collect()

    return explanations

# Explain QA predictions
lime_explanations_qa = explain_with_lime(bart_qa_model, qa_val_loader_v4, task="qa", num_samples=5)
shap_explanations_qa = explain_with_shap(bart_qa_model, qa_val_loader_v4, task="qa", num_samples=5)

# Explain triple predictions
lime_explanations_triple = explain_with_lime(bart_triple_model, triple_val_loader_v4, task="triple", num_samples=5)
shap_explanations_triple = explain_with_shap(bart_triple_model, triple_val_loader_v4, task="triple", num_samples=5)

# Save explanations
explainability_results = {
    "lime_qa": lime_explanations_qa,
    "shap_qa": shap_explanations_qa,
    "lime_triple": lime_explanations_triple,
    "shap_triple": shap_explanations_triple
}

explainability_path = os.path.join(CONFIG.BASE_PATH, "explainability_results_v4.json")
with open(explainability_path, "w") as f:
    json.dump(explainability_results, f)
print(f"Saved explainability results at {explainability_path}")

# Print sample explanations
print("\nSample LIME Explanation (QA):")
print(lime_explanations_qa[0])
print("\nSample SHAP Explanation (QA):")
print(shap_explanations_qa[0])
print("\nSample LIME Explanation (Triple):")
print(lime_explanations_triple[0])
print("\nSample SHAP Explanation (Triple):")
print(shap_explanations_triple[0])

Explaining qa predictions with LIME...
Explaining qa predictions with custom permutation SHAP...
Computing custom permutation SHAP for text: question: magnifier lamps are considered what type...
Computing custom permutation SHAP for text: question: who disagreed with idea of unilaterally ...
Computing custom permutation SHAP for text: question: what invading marauders caused havoc in ...
Computing custom permutation SHAP for text: question: when was non-electroluminescence discove...
Computing custom permutation SHAP for text: question: which three groups did early triassic pe...
Explaining triple predictions with LIME...
Explaining triple predictions with custom permutation SHAP...
Computing custom permutation SHAP for text: question: cantiere navale visentini symptoms conte...
Computing custom permutation SHAP for text: question: european peacebuilding liaison office sy...
Computing custom permutation SHAP for text: question: creator mundi symptoms context: creator ...
Computing cust

In [ ]:
# (Part 1): Quantitative Validation - Evaluate BART

# Evaluate BART on both QA and triple tasks
def evaluate_bart(model, val_loader, task: str = "qa"):
    print(f"Evaluating BART for {task}...")
    model.eval()
    bleu_scores, rouge_scores, bert_scores = [], [], []
    sample_outputs = []
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Evaluating {task}"):
            input_ids = batch["bart_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["bart_attention_mask"].to(CONFIG.DEVICE)
            references = batch["answer"]
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                num_beams=10,  # Reduced from 20 to 10
                temperature=0.5,
                no_repeat_ngram_size=2
            )
            generated_texts = [bart_tokenizer.decode(g_ids, skip_special_tokens=True) for g_ids in generated_ids]
            for gen, ref in zip(generated_texts, references):
                gen = normalize_text(gen)
                ref = normalize_text(ref)
                bleu = compute_bleu(gen, ref)
                rouge = compute_rouge_l(gen, ref)
                bert_f1 = compute_bertscore(gen, ref)
                bleu_scores.append(bleu)
                rouge_scores.append(rouge)
                bert_scores.append(bert_f1)
                sample_outputs.append((gen, ref))
            del input_ids, attention_mask, generated_ids
            torch.cuda.empty_cache()
    avg_bleu = np.mean(bleu_scores)
    avg_rouge = np.mean(rouge_scores)
    avg_bert = np.mean(bert_scores)
    print(f"BART {task} Evaluation:")
    print(f"Average BLEU: {avg_bleu:.4f}")
    print(f"Average ROUGE-L: {avg_rouge:.4f}")
    print(f"Average BERTScore F1: {avg_bert:.4f}")
    print(f"Sample Outputs (First 5) for {task}:")
    for gen, ref in sample_outputs[:5]:
        print(f"Generated: {gen}")
        print(f"Reference: {ref}\n")
    return avg_bleu, avg_rouge, avg_bert

# Evaluate BART on QA and triple tasks
bart_qa_bleu, bart_qa_rouge, bart_qa_bert = evaluate_bart(bart_qa_model, qa_val_loader_v4, task="qa")
bart_triple_bleu, bart_triple_rouge, bart_triple_bert = evaluate_bart(bart_triple_model, triple_val_loader_v4, task="triple")

Evaluating BART for qa...


Evaluating qa:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

BART qa Evaluation:
Average BLEU: 0.0000
Average ROUGE-L: 0.0000
Average BERTScore F1: 0.0000
Sample Outputs (First 5) for qa:
Generated: 
Reference: task lighting

Generated: 
Reference: president truman

Generated: 
Reference: barbarian

Generated: 
Reference: unanswerable

Generated: 
Reference: crocodylomorphs pterosaurs and dinosaurs

Evaluating BART for triple...


Streaming output truncated to the last 5000 lines.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoi

BART triple Evaluation:
Average BLEU: 0.0000
Average ROUGE-L: 0.0317
Average BERTScore F1: 0.8215
Sample Outputs (First 5) for triple:
Generated: iro
Reference: porto viro

Generated: us
Reference: brusselscapital region

Generated: tonton
Reference: littleton

Generated: abul
Reference: kabul

Generated: nzhenhen
Reference: shenzhen



In [ ]:
# (Part 2): Quantitative Validation - Evaluate DPR (Optimized for Version 3)

# Evaluate DPR on both QA and triple tasks at k=1, 5, 10
def evaluate_dpr_k(ctx_encoder, question_encoder, val_loader, candidates, k_values=[1, 5, 10], task: str = "qa"):
    ctx_encoder.eval()
    question_encoder.eval()
    mrr_k, precision_k = {k: [] for k in k_values}, {k: [] for k in k_values}
    eval_candidates = candidates[:500]  # Reduced from 1000 to 500
    print(f"Task: {task}, Using candidate pool size: {len(eval_candidates)}")
    candidate_inputs = ctx_tokenizer(eval_candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
    candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
    with torch.no_grad():
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Evaluating DPR {task}"):
            question_inputs = {
                "input_ids": batch["dpr_input_ids"].to(CONFIG.DEVICE),
                "attention_mask": batch["dpr_attention_mask"].to(CONFIG.DEVICE)
            }
            references = batch["answer"]
            question_embeddings = question_encoder(**question_inputs).pooler_output
            similarities = torch.matmul(question_embeddings, candidate_embeddings.T)
            rankings = torch.argsort(similarities, dim=1, descending=True)
            for i, (ranking, ref) in enumerate(zip(rankings, references)):
                ref_idx = eval_candidates.index(ref) if ref in eval_candidates else -1
                if ref_idx == -1:
                    continue
                for k in k_values:
                    top_k_indices = ranking[:k]
                    rank = (top_k_indices == ref_idx).nonzero(as_tuple=True)[0].item() + 1 if ref_idx in top_k_indices else len(eval_candidates)
                    mrr_k[k].append(1.0 / rank)
                    precision_k[k].append(1.0 if ref_idx in top_k_indices else 0.0)
            del question_inputs, similarities, rankings
            torch.cuda.empty_cache()
    results = {}
    for k in k_values:
        avg_mrr = np.mean(mrr_k[k])
        avg_precision = np.mean(precision_k[k])
        results[f"mrr_at_{k}"] = avg_mrr
        results[f"precision_at_{k}"] = avg_precision
        print(f"DPR Evaluation ({task}) at k={k}:")
        print(f"MRR@{k}: {avg_mrr:.4f}")
        print(f"Precision@{k}: {avg_precision:.4f}")
    return results

# Evaluate DPR on QA and triple tasks
dpr_qa_metrics = evaluate_dpr_k(ctx_encoder_qa, question_encoder_qa, qa_val_loader_v4, all_candidates, task="qa")
dpr_triple_metrics = evaluate_dpr_k(ctx_encoder_triple, question_encoder_triple, triple_val_loader_v4, all_candidates, task="triple")

Task: qa, Using candidate pool size: 500


Evaluating DPR qa: 100%|██████████| 4/4 [00:00<00:00,  4.12it/s]


DPR Evaluation (qa) at k=1:
MRR@1: 0.6963
Precision@1: 0.6957
DPR Evaluation (qa) at k=5:
MRR@5: 0.7973
Precision@5: 0.9130
DPR Evaluation (qa) at k=10:
MRR@10: 0.8044
Precision@10: 0.9565
Task: triple, Using candidate pool size: 500


Evaluating DPR triple: 100%|██████████| 654/654 [00:20<00:00, 32.07it/s]

DPR Evaluation (triple) at k=1:
MRR@1: 0.0930
Precision@1: 0.0912
DPR Evaluation (triple) at k=5:
MRR@5: 0.1330
Precision@5: 0.2084
DPR Evaluation (triple) at k=10:
MRR@10: 0.1417
Precision@10: 0.2749


In [ ]:
# (Part 3): Quantitative Validation - Evaluate DPR-based Ensemble

# Evaluate DPR-based Ensemble on both QA and triple tasks at k=1, 5, 10
def ensemble_evaluate_dpr_k(ctx_encoder, question_encoder, val_loader, candidates, k_values=[1, 5, 10], top_k: int = 30, task: str = "qa"):
    print(f"Evaluating DPR-based ensemble for {task}...")
    ctx_encoder.eval()
    question_encoder.eval()
    mrr_k, precision_k = {k: [] for k in k_values}, {k: [] for k in k_values}
    eval_candidates = candidates[:500]  # Reduced from 1000 to 500
    candidate_inputs = ctx_tokenizer(eval_candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
    candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
    with torch.no_grad():
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Ensemble Evaluating {task}"):
            question_inputs = {
                "input_ids": batch["dpr_input_ids"].to(CONFIG.DEVICE),
                "attention_mask": batch["dpr_attention_mask"].to(CONFIG.DEVICE)
            }
            references = batch["answer"]
            question_embeddings = question_encoder(**question_inputs).pooler_output
            similarities = torch.matmul(question_embeddings, candidate_embeddings.T)
            rankings = torch.argsort(similarities, dim=1, descending=True)
            top_k_indices = rankings[:, :top_k]  # Shape: (batch_size, top_k)
            batch_size = top_k_indices.size(0)
            for i in range(batch_size):
                top_k_candidate_indices = top_k_indices[i].cpu().numpy()
                top_k_candidates = [eval_candidates[idx] for idx in top_k_candidate_indices]
                ref = references[i]
                ref_idx = top_k_candidates.index(ref) if ref in top_k_candidates else -1
                if ref_idx == -1:
                    continue
                for k in k_values:
                    top_k_subset = top_k_indices[i, :k]
                    matches = torch.where(top_k_subset == top_k_candidate_indices[ref_idx])[0]
                    if matches.numel() > 0:
                        rank = matches[0].item() + 1
                    else:
                        rank = len(top_k_candidates)
                    mrr_k[k].append(1.0 / rank)
                    precision_k[k].append(1.0 if rank == 1 else 0.0)
            del question_inputs, similarities, rankings, top_k_indices
            torch.cuda.empty_cache()
    results = {}
    for k in k_values:
        avg_mrr = np.mean(mrr_k[k]) if mrr_k[k] else 0.0
        avg_precision = np.mean(precision_k[k]) if precision_k[k] else 0.0
        results[f"mrr_at_{k}"] = avg_mrr
        results[f"precision_at_{k}"] = avg_precision
        print(f"DPR-based Ensemble Evaluation ({task}) at k={k}:")
        print(f"MRR@{k}: {avg_mrr:.4f}")
        print(f"Precision@{k}: {avg_precision:.4f}")
    return results

# Evaluate ensemble on QA and triple tasks
ensemble_qa_metrics = ensemble_evaluate_dpr_k(ctx_encoder_qa, question_encoder_qa, qa_val_loader_v4, all_candidates, task="qa")
ensemble_triple_metrics = ensemble_evaluate_dpr_k(ctx_encoder_triple, question_encoder_triple, triple_val_loader_v4, all_candidates, task="triple")

Evaluating DPR-based ensemble for qa...


Ensemble Evaluating qa: 100%|██████████| 4/4 [00:00<00:00,  4.31it/s]


DPR-based Ensemble Evaluation (qa) at k=1:
MRR@1: 0.7364
Precision@1: 0.7273
DPR-based Ensemble Evaluation (qa) at k=5:
MRR@5: 0.8348
Precision@5: 0.7273
DPR-based Ensemble Evaluation (qa) at k=10:
MRR@10: 0.8409
Precision@10: 0.7273
Evaluating DPR-based ensemble for triple...


Ensemble Evaluating triple: 100%|██████████| 654/654 [00:20<00:00, 32.43it/s]

DPR-based Ensemble Evaluation (triple) at k=1:
MRR@1: 0.2383
Precision@1: 0.2120
DPR-based Ensemble Evaluation (triple) at k=5:
MRR@5: 0.3225
Precision@5: 0.2120
DPR-based Ensemble Evaluation (triple) at k=10:
MRR@10: 0.3380
Precision@10: 0.2120


In [ ]:
# (Part 4): Quantitative Validation - Save Results

# Save quantitative results
quantitative_results = {
    "bart_qa": {"bleu": bart_qa_bleu, "rouge": bart_qa_rouge, "bertscore": bart_qa_bert},
    "bart_triple": {"bleu": bart_triple_bleu, "rouge": bart_triple_rouge, "bertscore": bart_triple_bert},
    "dpr_qa": dpr_qa_metrics,
    "dpr_triple": dpr_triple_metrics,
    "ensemble_qa": ensemble_qa_metrics,
    "ensemble_triple": ensemble_triple_metrics
}

quantitative_path = os.path.join(CONFIG.BASE_PATH, "quantitative_results_v4.json")
with open(quantitative_path, "w") as f:
    json.dump(quantitative_results, f)
print(f"Saved quantitative results at {quantitative_path}")

Saved quantitative results at /content/drive/MyDrive/LJMU-Datasets/quantitative_results_v4.json


In [ ]:
# Qualitative Analysis (Human Assessment)

# Qualitative Analysis
def qualitative_analysis(model, val_loader, task: str = "qa", num_samples: int = 5):  # Reduced to 5 samples
    print(f"Performing qualitative analysis for {task}...")
    model.eval()
    samples = []
    samples_processed = 0

    with torch.no_grad():
        for batch in val_loader:
            if samples_processed >= num_samples:
                break
            input_ids = batch["bart_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["bart_attention_mask"].to(CONFIG.DEVICE)
            questions = batch["question"]
            references = batch["answer"]

            for i in range(input_ids.size(0)):
                if samples_processed >= num_samples:
                    break
                generated_ids = model.generate(
                    input_ids=input_ids[i:i+1],
                    attention_mask=attention_mask[i:i+1],
                    max_new_tokens=100,
                    num_beams=10,  # Reduced from 20 to 10
                    temperature=0.5,
                    no_repeat_ngram_size=2
                )
                generated_text = bart_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
                samples.append({
                    "question": questions[i],
                    "generated": generated_text,
                    "reference": references[i],
                    "coherence": "Incoherent" if generated_text.lower() in ["answer", "dasksk"] else "Coherent",
                    "reliability": "Reliable" if generated_text.lower() == references[i].lower() else "Unreliable",
                    "interpretability": "Interpretable" if generated_text.lower() not in ["answer", "dasksk"] else "Not Interpretable"
                })
                samples_processed += 1

            del input_ids, attention_mask, generated_ids
            torch.cuda.empty_cache()

    return samples

# Perform qualitative analysis for QA and triple tasks
qa_qualitative = qualitative_analysis(bart_qa_model, qa_val_loader_v4, task="qa", num_samples=5)
triple_qualitative = qualitative_analysis(bart_triple_model, triple_val_loader_v4, task="triple", num_samples=5)

# Print sample results
print("\nSample Qualitative Analysis (QA):")
for sample in qa_qualitative[:5]:
    print(f"Question: {sample['question']}")
    print(f"Generated: {sample['generated']}")
    print(f"Reference: {sample['reference']}")
    print(f"Coherence: {sample['coherence']}")
    print(f"Reliability: {sample['reliability']}")
    print(f"Interpretability: {sample['interpretability']}\n")

print("\nSample Qualitative Analysis (Triple):")
for sample in triple_qualitative[:5]:
    print(f"Question: {sample['question']}")
    print(f"Generated: {sample['generated']}")
    print(f"Reference: {sample['reference']}")
    print(f"Coherence: {sample['coherence']}")
    print(f"Reliability: {sample['reliability']}")
    print(f"Interpretability: {sample['interpretability']}\n")

# Save qualitative results (Version 4)
qualitative_results = {
    "qa": qa_qualitative,
    "triple": triple_qualitative
}

qualitative_path = os.path.join(CONFIG.BASE_PATH, "qualitative_analysis_v4.json")
with open(qualitative_path, "w") as f:
    json.dump(qualitative_results, f)
print(f"Saved qualitative analysis at {qualitative_path}")

Performing qualitative analysis for qa...
Performing qualitative analysis for triple...

Sample Qualitative Analysis (QA):
Question: magnifier lamps are considered what type of lighting?
Generated: 
Reference: task lighting
Coherence: Coherent
Reliability: Unreliable
Interpretability: Interpretable

Question: who disagreed with idea of unilaterally bombing north korean military?
Generated: 
Reference: president truman
Coherence: Coherent
Reliability: Unreliable
Interpretability: Interpretable

Question: what invading marauders caused havoc in balkans during 4th century
Generated: 
Reference: barbarian
Coherence: Coherent
Reliability: Unreliable
Interpretability: Interpretable

Question: when was non-electroluminescence discovered?
Generated: 
Reference: unanswerable
Coherence: Coherent
Reliability: Unreliable
Interpretability: Interpretable

Question: which three groups did early triassic period consist of?
Generated: 
Reference: crocodylomorphs pterosaurs and dinosaurs
Coherence: Cohe